In [ ]:
%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.103607935999989
GPU (s):
0.1044341439999954
GPU speedup over CPU: 29x


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import csv

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(tf.version.VERSION)

2.3.0


In [ ]:
BATCH_SIZE = 64
IMG_HEIGHT = 160
IMG_WIDTH = 160
IMG_SIZE = 160
train_dir = "./drive/My Drive/Datasets/FaceDataset"
CLASS_NAMES = np.array(['awake', 'sleep'])

In [ ]:
data_generator = ImageDataGenerator(rescale = 1./255.,
                                    validation_split=0.2)

In [ ]:
train_data_gen = data_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH),
                                               subset='training')
valid_data_gen = data_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH),
                                               subset='validation')

Found 1939 images belonging to 2 classes.
Found 484 images belonging to 2 classes.


In [ ]:
FEC_base = keras.models.load_model("./drive/My Drive/Models/FEC/FEC_base")
FEC_base.summary()

Model: "FEC_base"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
FaceNet_base (Functional)       (None, 8, 8, 896)    3406032     input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 8, 8, 512)    458752      FaceNet_base[0][0]               
__________________________________________________________________________________________________
dense1_block1_0_bn (BatchNormal (None, 8, 8, 512)    2048        conv2d[0][0]                     
___________________________________________________________________________________________

In [ ]:
FEC_base.trainable = False
model = keras.Sequential(layers=[
                          FEC_base,
                          layers.Dense(512, activation="relu"),
                          layers.Dense(2, activation='softmax')
                          ], name="FECNet_Classifier")
model.summary()

Model: "FECNet_Classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
FEC_base (Functional)        (None, 16)                5879264   
_________________________________________________________________
dense (Dense)                (None, 512)               8704      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 5,888,994
Trainable params: 9,730
Non-trainable params: 5,879,264
_________________________________________________________________


In [ ]:
num_train = 1939
num_valid = 484
steps_per_epoch = round(num_train)//BATCH_SIZE
validation_steps = round(num_valid)//BATCH_SIZE

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(train_data_gen,
                    callbacks=[callback],
                    steps_per_epoch=steps_per_epoch,
                    epochs=100,
                    validation_data=valid_data_gen,
                    validation_steps=validation_steps)
model.save("./drive/My Drive/Models/FECNet_sleep_19_8.h5")

In [ ]:
model = tf.keras.models.load_model("./drive/My Drive/Models/FECNet_sleep_19_8.h5")

In [ ]:
model.evaluate(train_data_gen, steps=steps_per_epoch)
model.evaluate(valid_data_gen, steps=validation_steps)

 9/30 [========>.....................] - ETA: 13:27 - loss: 0.1427 - accuracy: 0.9497